## Barrel Shifter

A Barrel shifter performs the operation `v << s` (or `v >> s` with flipped bits) where `s` is not constant.
An elegant implementation is the approach via cascaded multiplexer elements.

This `cshift` primitive is later instanced procedurally, iterating through the variable `i` 

In [1]:
from myirl.emulation.myhdl import *

@block
def cshift(q : Signal.Output, a : Signal, b : Signal, sbit : Signal, 
           msb : Signal,
           # These can take a signal or a fixed bool:
           asr, rotate,
           WRAP : bool):
    carry = Signal(bool()) # Carry bit
    u, v = [ Signal(bool()) for i in range(2) ]

    @always_comb
    def assign_carry():
        if asr: # arithmetic shift right
            carry.next = sbit & msb
        else:
            carry.next = 0

    @always_comb
    def assign():
        u.next = a & ~sbit
        
        if rotate == False:         
            if WRAP:
                v.next = carry
            else:
                v.next = b & sbit
        else:
            v.next = b & sbit
            
    @always_comb
    def assign_q():    
        q.next = u | v
        
    return instances()

### Shifter stage

In [2]:
@block
def shifter_stage(
    shifter,
    w_in : Signal,
    w_out : Signal.Output,
    msb : Signal,
    nmux : int, sbit : Signal, DATA_WIDTH : int, W_POWER : int, asr : bool, rotate : bool
):
    instances = []

    # Create signal array
    w = [ Signal(bool()) for i in range(DATA_WIDTH) ]
    wo = concat(*reversed(w))
    wi = [ w_in[i] for i in range(DATA_WIDTH) ]

    MUX_W = DATA_WIDTH // nmux

    for imux in range(nmux):
        tmp = imux * MUX_W
        print(imux)
        for i in range(tmp, tmp + MUX_W):
            j = i + MUX_W//2
            m = j % DATA_WIDTH
            instances.append(shifter(w[m], wi[m], wi[i], sbit, msb, asr, rotate,
                         j >= DATA_WIDTH ))


    @always_comb
    def assign():
        w_out.next = wo

    instances.append(assign)    

    return instances

### The barrel shifter implementation

In [3]:
@block
def barrel_shifter(shifter, clk : ClkSignal, ce : Signal, val : Signal, s : Signal, result : Signal.Output, \
                   rotate = False, W_POWER = 5 ):
    
    DATA_WIDTH = 2 ** W_POWER
    print("DATA WIDTH", DATA_WIDTH, "ROTATE", rotate)
        
    worker = [ val ]
    worker = worker + [ Signal(intbv()[DATA_WIDTH:]) for i in range(W_POWER) ]
    msb = val[DATA_WIDTH-1]

    sbit = [ s[i] for i in range(len(s))]
    
    shifter_stages = []
    for stage in range(W_POWER):
        K = W_POWER - stage - 1
        print("Stage %d" % stage)
        shifter_stages.append( \
                shifter_stage(shifter, worker[stage], worker[stage + 1], msb, 2 ** stage, sbit[K], \
                            DATA_WIDTH, W_POWER, False, rotate) \
                             )
        
    @always(clk.posedge)
    def assign():
        if ce == True:
            result.next = worker[W_POWER]
        
    return instances()

In [4]:
print(barrel_shifter.unparse())

Unparsing unit barrel_shifter


@block
def barrel_shifter(shifter, clk: ClkSignal, ce: Signal, val: Signal, s: Signal, result: Signal.Output, rotate=False, W_POWER=5):
    DATA_WIDTH = (2 ** W_POWER)
    print('DATA WIDTH', DATA_WIDTH, 'ROTATE', rotate)
    worker = [val]
    worker = (worker + [Signal(intbv()[DATA_WIDTH:]) for i in range(W_POWER)])
    msb = val[(DATA_WIDTH - 1)]
    sbit = [s[i] for i in range(len(s))]
    shifter_stages = []
    for stage in range(W_POWER):
        K = ((W_POWER - stage) - 1)
        print(('Stage %d' % stage))
        shifter_stages.append(shifter_stage(shifter, worker[stage], worker[(stage + 1)], msb, (2 ** stage), sbit[K], DATA_WIDTH, W_POWER, False, rotate))

    @always_(clk.posedge)
    def assign():
        (yield [If((ce == True)).Then(result.set(worker[W_POWER]))])
    return instances()



### Translate and test

In [5]:
W_POWER = 4

from myirl.test.common_test import gen_osc
import myirl

@block
def top_bs(shifter):
    clk = ClkSignal()
    ce = Signal(bool())
    val, result = [ Signal(intbv(0xaa00)[2 ** W_POWER:]) for i in range(2) ]
    s = Signal(intbv()[W_POWER:])
    
    inst = [
        barrel_shifter(shifter, clk, ce, val, s, result, False, W_POWER),
        gen_osc(clk, 2)
    ]
    
    
    TEST_VALUES = [
        (0xdead, 8, 0xad00),
        (0x8f01, 15, 0x8000),
    ]

    @instance
    def stim():
        for item in TEST_VALUES:
            ce.next = False
            s.next = item[1]
            val.next = item[0]
            yield(clk.posedge)
            ce.next = True
            yield(clk.posedge)
            yield(clk.posedge)

            print(result)
            assert result == _sequence.item[2] # XXX

    
    inst += [ stim ]
    return inst
 
def test(shifter_element):    
    return top_bs(shifter_element)

design = test(cshift)
tmp = top_bs.ctx.path_prefix

DATA WIDTH 16 ROTATE False
Stage 0
0
 Insert unit cshift_s1_s1_s1_s1_s1_0_0_0 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_0 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_0 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_0 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_0 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_0 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_0 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_0 
 Module top_top_bs: Existing implementation cshift, rename to cshift_1 
 Insert unit cshift_s1_s1_s1_s1_s1_0_0_1 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_1 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_1 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_1 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_1 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_1 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_1 
 DEBUG: Cached unit cshift_s1_s1_s1_s1_s1_0_0_1 
 Insert unit shifter_stage__wrapped_wrapper_s16_s16_s1_1_s1_16_4_0_0 
Stage 1
 Module top_top_bs: Existing implementation

In [6]:
from myirl import targets
from myirl.test.common_test import run_ghdl

f = design.elab(targets.VHDL, elab_all = True)
run_ghdl(f, design, vcdfile = 'bs.vcd', debug = True)

 DEBUG: Writing 'shifter_stage_3' to file /tmp/shifter_stage_3.vhdl 
Finished _elab in 0.0073 secs
 DEBUG: Writing 'shifter_stage_2' to file /tmp/shifter_stage_2.vhdl 
Finished _elab in 0.0077 secs
 DEBUG: Writing 'shifter_stage_1' to file /tmp/shifter_stage_1.vhdl 
Finished _elab in 0.0276 secs
 DEBUG: Writing 'cshift_1' to file /tmp/cshift_1.vhdl 
Finished _elab in 0.0032 secs
 DEBUG: Writing 'cshift' to file /tmp/cshift.vhdl 
Finished _elab in 0.0031 secs
 DEBUG: Writing 'shifter_stage' to file /tmp/shifter_stage.vhdl 
Finished _elab in 0.0088 secs
 DEBUG: Writing 'barrel_shifter' to file /tmp/barrel_shifter.vhdl 
Finished _elab in 0.0042 secs
 DEBUG: Writing 'top_bs' to file /tmp/top_bs.vhdl 
Finished _elab in 0.0170 secs
DEBUG Creating library file /tmp/module_defs.vhdl
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====
analyze /home/testing/.local/lib/python3.8/site-packages/myirl-0.0.0-py3.8-linux-x86_64.egg/myirl/targets/../test/vhdl/txt_util.vhdl
analyze /h

0

In [7]:
!cat {tmp}/barrel_shifter.vhdl

-- File generated from /usr/local/lib/python3.8/runpy.py
-- (c) 2016-2021 section5.ch
-- Modifications may be lost

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity barrel_shifter is
    port (
        clk : in std_ulogic;
        ce : in std_ulogic;
        val : in unsigned(15 downto 0);
        s : in unsigned(3 downto 0);
        result : out unsigned(15 downto 0)
    );
end entity barrel_shifter;

architecture MyIRL of barrel_shifter is
    -- Local type declarations
    -- Signal declarations
    signal worker4 : unsigned(15 downto 0);
    signal worker1 : unsigned(15 downto 0);
    signal worker2 : unsigned(15 downto 0);
    signal worker3 : unsigned(15 downto 0);
begin
    
assign:
    process(clk)
    begin
        if rising_edge(clk) then
            if (ce = '1') then
                result <= worker4;
            end if;
        end if;
    end process;
    -- Instance shifter_